In [1]:
from environs import Env

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts.prompt import PromptTemplate


In [2]:
# import env config file
env = Env()
env.read_env('../.env', recurse=False)

True

In [16]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [41]:
def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings(openai_api_key=env("OPENAI_TOKEN"))
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [44]:
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI(openai_api_key=env("OPENAI_TOKEN"))
    # llm = HuggingFaceHub(
    #     repo_id="google/flan-t5-xxl",
    #     model_kwargs={"temperature": 0.5, "max_length": 512},
    # )
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    conversation_chain = ConversationChain.from_llm(
        llm=llm, memory=memory
    )
    return conversation_chain

In [14]:
with open('../docs/traing_gpt.txt', 'r') as f:
    system_prompt = f.read()

In [39]:
# get the text chunks
text_chunks = get_text_chunks(system_prompt)

In [43]:
# create vector store
vectorstore = get_vectorstore(text_chunks)

In [45]:
# create conversation chain
state = get_conversation_chain(vectorstore)

In [48]:
state('А сколько они стоят')

{'question': 'А сколько они стоят',
 'chat_history': [HumanMessage(content='Как дела?'),
  AIMessage(content='Добрый день! Я всего лишь бот, поэтому не могу иметь настроения или делиться своими эмоциями. Но я готов ответить на ваши вопросы и помочь вам с курсами по нейропсихологии. Какая информация вас интересует?'),
  HumanMessage(content='Привет'),
  AIMessage(content="У нас есть различные курсы по нейропсихологии, которые охватывают от начальных до продвинутых уровней. Вы можете начать с 'Основ нейропсихологии' или погрузиться в специализированные темы, такие как 'Нейропсихология обучения' и 'Эмоциональный интеллект'. Что вас больше всего интересует?"),
  HumanMessage(content='А сколько они стоят'),
  AIMessage(content='Цены на курсы варьируются в зависимости от их длительности и уровня сложности. Например, Основы нейропсихологии стоит 700р, а курс Нейропсихология обучения - 700р. Все актуальные цены вы можете найти на нашем сайте. Также мы регулярно предлагаем скидки и специальные 

In [50]:
state.

[]

In [4]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Как меня зовут"}
  ]
)
print(response.choices[0].message.content)

К сожалению, я не могу узнать, как вас зовут, так как я всего лишь чат-бот.
